# Encontrando insights de la UEFA Champions League

La Liga de Campeones de la UEFA, a menudo llamada Liga de Campeones, es una competencia anual de fútbol por excelencia que cautiva a los fanáticos de todo el mundo. Establecida en 1955 como la Copa de Clubes Campeones de Europa, evolucionó a la Liga de Campeones de la UEFA en 1992, ampliando su atractivo. El formato moderno cuenta con 32 equipos de clubes de primer nivel seleccionados en función de su desempeño en la liga nacional, lo que aumenta la intriga.  

Este evento electrizante trasciende los deportes y se convierte en una celebración de la unidad, la cultura y el orgullo nacional. Los fanáticos, vestidos con los colores de sus países, crean una atmósfera eléctrica, lo que hace que el torneo sea tanto sobre los espectadores como sobre los jugadores. Financieramente, la Liga de Campeones es un salvavidas para los clubes, ya que aumenta los ingresos y ofrece oportunidades transformadoras. Sin embargo, genera debates sobre las disparidades de riqueza en el fútbol europeo.  

La Liga de Campeones es sinónimo de rivalidades históricas, triunfos de los desvalidos y brillantez individual. Para los jugadores, representa la cima de su carrera, mientras que para los fanáticos, es un fenómeno cultural. El himno y los rituales icónicos enriquecen la experiencia futbolística. En 200 palabras, la UEFA Champions League es el epítome de la excelencia del fútbol europeo, que ofrece momentos inolvidables, recompensas económicas y un impacto cultural único, con 32 clubes de primer nivel que se suman a su atractivo.

**Nombres de las tablas o ficheros a utilizar**:  
[uefa_2020.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EQuW8d16tZJKpYZN6jR480sBgc1IYypAw9hGHKQARS560g?e=7TybBU)  
[uefa_2021.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ESG8YBrjD3tBky3aGT0MwIABNt_PQxFVVYVlWMnZYUmV8g?e=VrYHtX)  
[uefa_2022.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/Ecf8O8U8sxJLoW9Bkd2ZVzgBb_gZc35mNcprnR9FMlzliQ?e=Vn3a6v)  
- Todas las tablas tienen las mismas columnas y data types

| Column | Definition | Data type |
|--------|------------|-----------|
| `STAGE`| Stage of the March | `VARCHAR(50)` |
| `DATE` | When the match occurred. | `DATE` |
| `PENS` | Did the match end with penalty | `VARCHAR(50)` |
| `PENS_HOME_SCORE` | In case of penalty, score by home team | `VARCHAR(50)` |
| `PENS_AWAY_SCORE` | In case of penalty, score by away team | `VARCHAR(50)` |
| `TEAM_NAME_HOME` | Team home name | `VARCHAR(50)` |
| `TEAM_NAME_AWAY`| Team away  name | `VARCHAR(50)` |
| `TEAM_HOME_SCORE` | Team home score | `NUMBER` |
| `TEAM_AWAY_SCORE` | Team away score | `NUMBER` |
| `POSSESSION_HOME` | Ball possession for the home team | `FLOAT` |
| `POSSESSION_AWAY` | Ball possession for the away team | `FLOAT` |
| `TOTAL_SHOTS_HOME` | Number of shots by the home team | `NUMBER` |
| `TOTAL_SHOTS_AWAY` | Number of shots by the away team | `NUMBER`
| `SHOTS_ON_TARGET_HOME` | Total shot for home team | `FLOAT` |
| `SHOTS_ON_TARGET_AWAY` | Total shot for away team | `FLOAT` |
| `DUELS_WON_HOME` | duel win possession of ball - for home team | `NUMBER` |
| `DUELS_WON_AWAY` | duel win possession of ball - for away team | `NUMBER`
| `PREDICTION_TEAM_HOME_WIN` | Probability of home team to win | `FLOAT` |
| `PREDICTION_DRAW` | Probability of draw | `FLOAT` |
| `PREDICTION_TEAM_AWAY_WIN` | Probability of away team to win | `FLOAT` |
| `LOCATION` | Stadium where the match was held | `VARCHAR(50)` |

In [17]:
!pip install pyspark

In [18]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("CargarCSV").getOrCreate()

In [19]:
df_2020 = spark.read.csv("uefa_2020.csv", header=True, inferSchema=True)
df_2021 = spark.read.csv("uefa_2021.csv", header=True, inferSchema=True)
df_2022 = spark.read.csv("uefa_2022.csv", header=True, inferSchema=True)

### Requerimientos:  
-Puedes utilizar SQL o PySpark

1. Encuentra los 3 equipos que anotaron más goles jugando en su estadio en la UEFA Champions League 2020-21. El resultado debe contener dos columnas: TEAM_NAME_HOME y TEAM_HOME_SCORE ordenadas en orden descendente de TEAM_HOME_SCORE. Guarda la consulta (o el notebook) como TEAM_HOME_WITH_MOST_GOALS.

In [27]:
TEAM_HOME_WITH_MOST_GOALS = df_2020.groupBy("TEAM_NAME_HOME").agg(F.sum(F.col("TEAM_HOME_SCORE")).alias("TEAM_HOME_SCORE")).orderBy("TEAM_HOME_SCORE", ascending=False).limit(3)
TEAM_HOME_WITH_MOST_GOALS.show(truncate=False)

+-----------------+---------------+
|TEAM_NAME_HOME   |TEAM_HOME_SCORE|
+-----------------+---------------+
| Manchester City |15             |
| Real Madrid     |14             |
| Bayern Munich   |13             |
+-----------------+---------------+



Creamos un filtro donde ordenamos los goles marcados en casa (TEAM_HOME_SCORE) descendentemente para ver el top 3

2. Encuentra el equipo con posesión mayoritaria la mayor cantidad de veces durante la UEFA Champions League 2021-22. El resultado debe incluir dos columnas: TEAM_NAME y GAME_COUNT, que es la cantidad de veces que el equipo tuvo posesión mayoritaria durante un partido de fútbol. Guarda esta consulta (o el notebook) como TEAM_WITH_MAJORITY_POSSESSION

In [25]:
import pyspark.sql.functions as F

# primero spliteamos los valores del porcentaje de posesión para poder comparar sus valores
posesion_home = F.split(F.col("POSSESSION_HOME"), "%")[0]
posesion_away = F.split(F.col("POSSESSION_AWAY"), "%")[0]

# filtramos las líneas en las que cada equipo haya ganado la posesión, sea visitante o local
df_posesion_won_home = df_2021.where((posesion_home > posesion_away)).groupBy(F.col("TEAM_NAME_HOME").alias("TEAM_NAME")).agg(F.count(F.col("POSSESSION_HOME")).alias("GAME_COUNT"))
df_posesion_won_away = df_2021.where((posesion_away > posesion_home)).groupBy(F.col("TEAM_NAME_AWAY").alias("TEAM_NAME")).agg(F.count(F.col("POSSESSION_AWAY")).alias("GAME_COUNT"))

# juntamos ambos dataframes para tener toda la información junta
df_posesion_joined = df_posesion_won_home.union(df_posesion_won_away)

# sumamos todos los partidos en los que un equipo ha ganado en posesión
df_posesion_joined = df_posesion_joined.groupBy(F.col("TEAM_NAME")).agg(F.sum(F.col("GAME_COUNT")).alias("GAME_COUNT"))

# asignamos el valor a una variable
TEAM_WITH_MAJORITY_POSSESSION = df_posesion_joined.orderBy(F.col("GAME_COUNT").desc())

# mostramos el mejor (top 1)
TEAM_WITH_MAJORITY_POSSESSION.show(1)


+---------+----------+
|TEAM_NAME|GAME_COUNT|
+---------+----------+
|Liverpool|         9|
+---------+----------+
only showing top 1 row



El resultado nos muestra el equipo con más partidos en los que ha ganado en posesión, sin importar si es visitante o local. Para ello he creado 2 dataframes filtrando por visitante y por local y los he juntado en uno solo para después sumarlo todo junto

3. Encuentra la lista de equipos de cada fase del juego que ganaron el duelo en un partido pero terminaron perdiendo el juego en el Campeonato de la UEFA 2022-23. El resultado debe contener dos columnas: STAGE y TEAM_LOST. Guarda la consulta (o el notebook) como TEAM_WON_DUEL_LOST_GAME_STAGE_WISE.

In [2]:
from google.colab import files

uploaded = files.upload()

Saving uefa_2020.csv to uefa_2020.csv
Saving uefa_2021.csv to uefa_2021.csv
Saving uefa_2022.csv to uefa_2022.csv


In [16]:
import pyspark.sql.functions as F

# primero spliteamos los valores del porcentaje de duelos ganados para poder comparar sus valores
duels_won_home = F.split(F.col("DUELS_WON_HOME"), "%")[0]
duels_won_away = F.split(F.col("DUELS_WON_AWAY"), "%")[0]

# filtramos las líneas en las que los duelos hayan sido ganados por un equipo, y ese equipo haya perdido el partido
df_duels_home_lost = df_2022.where((duels_won_home > duels_won_away) & (F.col("TEAM_HOME_SCORE") < F.col("TEAM_AWAY_SCORE")))
df_duels_away_lost = df_2022.where((duels_won_away > duels_won_home) & (F.col("TEAM_HOME_SCORE") > F.col("TEAM_AWAY_SCORE")))

# juntamos ambos dataframes para tener toda la información junta
df_duels_won_lost_joined = df_duels_home_lost.union(df_duels_away_lost)

# creamos una columna llamada TEAM_LOST indicando cuál es el equipo perdedor
df_duels_won_lost_joined = df_duels_won_lost_joined.withColumn(
    "TEAM_LOST",
    F.when(F.col("TEAM_HOME_SCORE") > F.col("TEAM_AWAY_SCORE"), F.col("TEAM_NAME_AWAY"))
     .when(F.col("TEAM_HOME_SCORE") < F.col("TEAM_AWAY_SCORE"), F.col("TEAM_NAME_HOME"))
)

# guardamos los datos en una variable
TEAM_WON_DUEL_LOST_GAME_STAGE_WISE = df_duels_won_lost_joined.select("STAGE", "TEAM_LOST")

# mostramos el resultado
TEAM_WON_DUEL_LOST_GAME_STAGE_WISE.show(truncate=False)

+-------------------------+-----------------+
|STAGE                    |TEAM_LOST        |
+-------------------------+-----------------+
| Group stage: Matchday 1 |Chelsea          |
| Group stage: Matchday 1 |K�benhavn        |
| Group stage: Matchday 1 |Juventus         |
| Group stage: Matchday 1 |Maccabi Haifa    |
| Group stage: Matchday 1 |Rangers          |
| Group stage: Matchday 1 |Liverpool        |
| Group stage: Matchday 1 |Porto            |
| Group stage: Matchday 1 |Bayer Leverkusen |
| Group stage: Matchday 1 |Viktoria Plze?   |
| Group stage: Matchday 1 |Marseille        |
| Group stage: Matchday 2 |Ajax             |
| Group stage: Matchday 2 |Atletico Madrid  |
| Group stage: Matchday 2 |Barcelona        |
| Group stage: Matchday 2 |Tottenham Hotspur|
| Group stage: Matchday 2 |RB Leipzig       |
| Group stage: Matchday 3 |Rangers          |
| Group stage: Matchday 3 |Atletico Madrid  |
| Group stage: Matchday 3 |Bayer Leverkusen |
| Group stage: Matchday 3 |Viktori

El resultado nos muestra todos los equipos que han ganado en duelos, y han terminado perdiendo el partido.

Para ello, también he creado 2 dataframes istintos filtrando por visitante y local, y los he unido todos en uno para obtener los datos de todos los equipos que cumpliesen dicha condición